## How to Plot a Route on a Map

notebook of the article below  
https://towardsdatascience.com/how-to-plot-a-route-on-a-map-fb900a7f6605  
Please note that the API key is missing, you must register on the website indicated in the article (credit card required)  

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
def get_lat_long_from_address(address):       
    locator = Nominatim(user_agent='myGeocoder')
    location = locator.geocode(address)
    return location.latitude, location.longitude

In [ ]:
# example
address = 'Zeeweg 94, 2051 EC Overveen'
get_lat_long_from_address(address)

In [ ]:
import requests

In [ ]:
def get_directions_response(lat1, long1, lat2, long2, mode='drive'):
    url = "https://route-and-directions.p.rapidapi.com/v1/routing"
    key = "YOUR-API-KEY"
    host = "route-and-directions.p.rapidapi.com"
    headers = {"X-RapidAPI-Key": key, "X-RapidAPI-Host": host}
    querystring = {"waypoints":f"{str(lat1)},{str(long1)}|{str(lat2)},{str(long2)}","mode":mode}
    response = requests.request("GET", url, headers=headers, params=querystring)
    return response

In [ ]:
response = get_directions_response(52.4013, 4.5425, 52.402, 4.5426)

In [ ]:
response.json()

In [ ]:
import folium
import pandas as pd

In [ ]:
def create_map(response):
    # use the response
    mls = response.json()['features'][0]['geometry']['coordinates']
    points = [(i[1], i[0]) for i in mls[0]]   
    m = folium.Map()  
    
    # add marker for the start and ending points
    for point in [points[0], points[-1]]:
        folium.Marker(point).add_to(m) 
        
    # add the lines
    folium.PolyLine(points, weight=5, opacity=1).add_to(m)   
    # create optimal zoom
    df = pd.DataFrame(mls[0]).rename(columns={0:'Lon', 1:'Lat'})[['Lat', 'Lon']]
    sw = df[['Lat', 'Lon']].min().values.tolist()
    ne = df[['Lat', 'Lon']].max().values.tolist()
    m.fit_bounds([sw, ne])
    return m

In [ ]:
m = create_map(response)

In [ ]:
m